In [82]:
import pandas as pd

In [83]:
# 탑리뷰어 정보 DF로 가져오기
topReviews_path = '/Users/ryeongjoo/Desktop/workspace/semi_prj/topReviews.csv'
top_df = pd.read_csv(topReviews_path, header=0, encoding='UTF-8')

/var/folders/mn/w592whfs2tb4g8s658288fb00000gn/T/ipykernel_49290/3744978623.py:3: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  top_df = pd.read_csv(topReviews_path, header=0, encoding='UTF-8')


In [84]:
# 작성자, 주문 날짜, 리뷰 작성 날짜, 리뷰 내용, 리뷰당 ‘도움이 돼요’ 개수, 한달 이상 사용, 재구매 여부, 사진 개수, 제품 구매 방법, 리뷰 고유번호, 평점(긍부정)
# 리뷰 쓴 주기, 리뷰 길이, 총 ‘도움이 돼요’, 리뷰당 ‘도움이 돼요’ (평균), 체험단 리뷰와 일반 리뷰 차이 (리뷰 길이 편차)

# gdasSeq  :  리뷰 고유번호
# gdasScrVal: 평점 (2,4,6,8,10)
# dispRegDate : 리뷰 업로드 날짜
# gdasCont : 리뷰 본문
# ordDate: 상품 주문 날짜
# photoList: 사진 리스트 (사진 없으면 null)
# recommCnt: 도움이 돼요 개수
# renewUsed1mmGdasYn: 한달이상 사용
# firstGdasYn: 첫 구매 여부 (재구매면 “N” 아니면 “Y”)
# mbrNo: 리뷰를 작성한 회원의 고유 식별자
# gdasSctCd : 제품 구매 방법 (10 - 온라인, 60 - 매장, 50 - 체험단, 70 - 선물)

In [85]:
# 필요한 칼럼만 담은 DF
my_top_df = top_df[['mbrNo','ordDate','dispRegDate','gdasCont','recommCnt','renewUsed1mmGdasYn','firstGdasYn', 'gdasSeq', 'gdasScrVal','gdasSctCd' ]]
print(my_top_df.head())

            mbrNo     ordDate dispRegDate  \
0  M0000004535694  2023.01.23  2023.04.21   
1  M0000004535694  2023.01.23  2023.04.21   
2  M0000004535694  2023.01.23  2023.04.21   
3  M0000004535694         NaN  2023.04.20   
4  M0000004535694  2023.01.23  2023.04.20   

                                            gdasCont  recommCnt  \
0  제가 최근에 올리브영에서 가장 많이\r\n구매를 했던 제품중 하나입니다ㅎ\r\n신랑...        NaN   
1  허브쏠트 믹스넛은 견과류구성은\r\n맘에 드는데 넘짜요ㅠ\r\n그래도 1+1 행사를...        NaN   
2  아침에 주로 마시고 있는 사과당근맛\r\n이너주스입니다ㅎ\r\n제가 5년을 넘게 마...        NaN   
3  신랑을 줄려고 신청을 했어요~\r\n다비도프 향수는 프랑스에서 만들어\r\n졌고 향...        NaN   
4  팽이버섯 유산균 발효액이 들어가서\r\n꾸준하게 마시면 화장실 가는게\r\n편해지네요ㅎㅎ        NaN   

  renewUsed1mmGdasYn firstGdasYn   gdasSeq  gdasScrVal  gdasSctCd  
0                  Y           N  19025253          10         60  
1                  Y           N  19025210           8         60  
2                  Y           N  19025179          10         60  
3                  N           Y  19017693          10

In [86]:
# 리뷰 길이 칼럼 추가
my_top_df = my_top_df.dropna(subset=['gdasCont']) # NaN값 4개 드랍
my_top_df['cont_length'] = my_top_df['gdasCont'].apply(lambda x: len(str(x)))

print(my_top_df.head())

            mbrNo     ordDate dispRegDate  \
0  M0000004535694  2023.01.23  2023.04.21   
1  M0000004535694  2023.01.23  2023.04.21   
2  M0000004535694  2023.01.23  2023.04.21   
3  M0000004535694         NaN  2023.04.20   
4  M0000004535694  2023.01.23  2023.04.20   

                                            gdasCont  recommCnt  \
0  제가 최근에 올리브영에서 가장 많이\r\n구매를 했던 제품중 하나입니다ㅎ\r\n신랑...        NaN   
1  허브쏠트 믹스넛은 견과류구성은\r\n맘에 드는데 넘짜요ㅠ\r\n그래도 1+1 행사를...        NaN   
2  아침에 주로 마시고 있는 사과당근맛\r\n이너주스입니다ㅎ\r\n제가 5년을 넘게 마...        NaN   
3  신랑을 줄려고 신청을 했어요~\r\n다비도프 향수는 프랑스에서 만들어\r\n졌고 향...        NaN   
4  팽이버섯 유산균 발효액이 들어가서\r\n꾸준하게 마시면 화장실 가는게\r\n편해지네요ㅎㅎ        NaN   

  renewUsed1mmGdasYn firstGdasYn   gdasSeq  gdasScrVal  gdasSctCd  cont_length  
0                  Y           N  19025253          10         60           81  
1                  Y           N  19025210           8         60           78  
2                  Y           N  19025179          10         60           75  
3 

In [87]:
# 리뷰 주기 칼럼
# 리뷰 작성 날짜 - 구매일자

# 날짜형으로 변환
my_top_df['dispRegDate'] = pd.to_datetime(my_top_df['dispRegDate'])
my_top_df['ordDate'] = pd.to_datetime(my_top_df['ordDate'])

# NaN이 아닌 행 선택
subset = my_top_df[my_top_df['ordDate'].notnull() ]

# 두 날짜의 차이를 계산하여 새로운 칼럼에 추가
my_top_df.loc[subset.index, 'or_diff'] = my_top_df['dispRegDate'] - my_top_df['ordDate']

# 결과 출력
print(my_top_df.head())

            mbrNo    ordDate dispRegDate  \
0  M0000004535694 2023-01-23  2023-04-21   
1  M0000004535694 2023-01-23  2023-04-21   
2  M0000004535694 2023-01-23  2023-04-21   
3  M0000004535694        NaT  2023-04-20   
4  M0000004535694 2023-01-23  2023-04-20   

                                            gdasCont  recommCnt  \
0  제가 최근에 올리브영에서 가장 많이\r\n구매를 했던 제품중 하나입니다ㅎ\r\n신랑...        NaN   
1  허브쏠트 믹스넛은 견과류구성은\r\n맘에 드는데 넘짜요ㅠ\r\n그래도 1+1 행사를...        NaN   
2  아침에 주로 마시고 있는 사과당근맛\r\n이너주스입니다ㅎ\r\n제가 5년을 넘게 마...        NaN   
3  신랑을 줄려고 신청을 했어요~\r\n다비도프 향수는 프랑스에서 만들어\r\n졌고 향...        NaN   
4  팽이버섯 유산균 발효액이 들어가서\r\n꾸준하게 마시면 화장실 가는게\r\n편해지네요ㅎㅎ        NaN   

  renewUsed1mmGdasYn firstGdasYn   gdasSeq  gdasScrVal  gdasSctCd  \
0                  Y           N  19025253          10         60   
1                  Y           N  19025210           8         60   
2                  Y           N  19025179          10         60   
3                  N           Y  19017693          10  

In [88]:
# 리뷰어별 총 도움이 돼요 개수

# NaN이 아닌 행 선택
subset = my_top_df[my_top_df['recommCnt'].notnull() ]

# 두 날짜의 차이를 계산하여 새로운 칼럼에 추가
recommSum_df = my_top_df.groupby('mbrNo')['recommCnt'].sum().reset_index().rename(columns={'recommCnt': 'recommCnt_sum'})
my_top_df = pd.merge(my_top_df, recommSum_df, on='mbrNo')

print(my_top_df.head())

            mbrNo    ordDate dispRegDate  \
0  M0000004535694 2023-01-23  2023-04-21   
1  M0000004535694 2023-01-23  2023-04-21   
2  M0000004535694 2023-01-23  2023-04-21   
3  M0000004535694        NaT  2023-04-20   
4  M0000004535694 2023-01-23  2023-04-20   

                                            gdasCont  recommCnt  \
0  제가 최근에 올리브영에서 가장 많이\r\n구매를 했던 제품중 하나입니다ㅎ\r\n신랑...        NaN   
1  허브쏠트 믹스넛은 견과류구성은\r\n맘에 드는데 넘짜요ㅠ\r\n그래도 1+1 행사를...        NaN   
2  아침에 주로 마시고 있는 사과당근맛\r\n이너주스입니다ㅎ\r\n제가 5년을 넘게 마...        NaN   
3  신랑을 줄려고 신청을 했어요~\r\n다비도프 향수는 프랑스에서 만들어\r\n졌고 향...        NaN   
4  팽이버섯 유산균 발효액이 들어가서\r\n꾸준하게 마시면 화장실 가는게\r\n편해지네요ㅎㅎ        NaN   

  renewUsed1mmGdasYn firstGdasYn   gdasSeq  gdasScrVal  gdasSctCd  \
0                  Y           N  19025253          10         60   
1                  Y           N  19025210           8         60   
2                  Y           N  19025179          10         60   
3                  N           Y  19017693          10  

In [94]:
# 리뷰어별 리뷰 개수당 평균 도움이 돼요 개수

# 리뷰어별 작성 리뷰 개수
reviewCnt_df = my_top_df.groupby('mbrNo')['gdasCont'].count().reset_index().rename(columns={'gdasCont': 'gdasContt_cnt'})
my_top_df = pd.merge(my_top_df, reviewCnt_df, on='mbrNo')

# 나누기
my_top_df['recomm_per_rv'] = my_top_df['recommCnt_sum'] / my_top_df['gdasContt_cnt']

print(my_top_df.head())

            mbrNo    ordDate dispRegDate  \
0  M0000004535694 2023-01-23  2023-04-21   
1  M0000004535694 2023-01-23  2023-04-21   
2  M0000004535694 2023-01-23  2023-04-21   
3  M0000004535694        NaT  2023-04-20   
4  M0000004535694 2023-01-23  2023-04-20   

                                            gdasCont  recommCnt  \
0  제가 최근에 올리브영에서 가장 많이\r\n구매를 했던 제품중 하나입니다ㅎ\r\n신랑...        NaN   
1  허브쏠트 믹스넛은 견과류구성은\r\n맘에 드는데 넘짜요ㅠ\r\n그래도 1+1 행사를...        NaN   
2  아침에 주로 마시고 있는 사과당근맛\r\n이너주스입니다ㅎ\r\n제가 5년을 넘게 마...        NaN   
3  신랑을 줄려고 신청을 했어요~\r\n다비도프 향수는 프랑스에서 만들어\r\n졌고 향...        NaN   
4  팽이버섯 유산균 발효액이 들어가서\r\n꾸준하게 마시면 화장실 가는게\r\n편해지네요ㅎㅎ        NaN   

  renewUsed1mmGdasYn firstGdasYn   gdasSeq  gdasScrVal  gdasSctCd  \
0                  Y           N  19025253          10         60   
1                  Y           N  19025210           8         60   
2                  Y           N  19025179          10         60   
3                  N           Y  19017693          10  